<a href="https://colab.research.google.com/github/deviknitkkr/ML-Models/blob/main/LSTM-base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install tf-nightly-2.0-preview
import pandas as pd
import plotly.graph_objs as go
import plotly.offline as pyo
import yfinance as yf
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
import prophet 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
ticker = yf.Ticker("SBIN.NS")
time_period = '20mo'
df = ticker.history(period=time_period)

In [4]:
fig = go.Figure(data=[go.Candlestick(x=df.index,
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'])])
fig.add_trace(go.Scatter(x=df.index, y=df['Close'].rolling(window=1).mean(), name='SMA 1'))

fig.update_layout(title='Apple Stock Data (Last {})'.format(time_period),
                  yaxis_title='Price ($)',
                  xaxis_rangeslider_visible=True)
pyo.iplot(fig)

In [7]:
# Prepare the data for the LSTM model
data = df['Close'].values.reshape(-1,1)
scaler = MinMaxScaler(feature_range=(0, 1))
data = scaler.fit_transform(data)

# Split the data into training and testing sets
train_size = int(len(data) * 0.8)
test_size = len(data) - train_size
train_data = data[0:train_size,:]
test_data = data[train_size:len(data),:]

# Create the training and testing data sets
look_back = 30
trainX, trainY = [], []
for i in range(len(train_data)-look_back-1):
    a = train_data[i:(i+look_back), 0]
    trainX.append(a)
    trainY.append(train_data[i + look_back, 0])

trainX = np.array(trainX)
trainY = np.array(trainY)

testX, testY = [], []
for i in range(len(test_data)-look_back-1):
    a = test_data[i:(i+look_back), 0]
    testX.append(a)
    testY.append(test_data[i + look_back, 0])
testX = np.array(testX)
testY = np.array(testY)

# Create the LSTM model
model = Sequential()
model.add(LSTM(50, input_shape=(look_back, 1)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the LSTM model
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)

# Make predictions using the LSTM model
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

# Invert the predictions back to their original scale
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform(trainY.reshape(-1,1))
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform(testY.reshape(-1,1))

# Create a new dataframe with the actual stock prices and the predicted prices
trainPredictPlot = np.empty_like(data)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
testPredictPlot = np.empty_like(data)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(data)-1, :] = testPredict
actualPlot = scaler.inverse_transform(data)

df_pred = pd.DataFrame({'Actual': actualPlot.reshape(-1), 'Train Predicted': trainPredictPlot.reshape(-1), 'Test Predicted': testPredictPlot.reshape(-1)})


Epoch 1/100
301/301 - 4s - loss: 0.0110 - 4s/epoch - 14ms/step
Epoch 2/100
301/301 - 1s - loss: 0.0041 - 1s/epoch - 5ms/step
Epoch 3/100
301/301 - 2s - loss: 0.0033 - 2s/epoch - 5ms/step
Epoch 4/100
301/301 - 1s - loss: 0.0027 - 1s/epoch - 5ms/step
Epoch 5/100
301/301 - 1s - loss: 0.0026 - 1s/epoch - 4ms/step
Epoch 6/100
301/301 - 1s - loss: 0.0024 - 1s/epoch - 5ms/step
Epoch 7/100
301/301 - 1s - loss: 0.0022 - 1s/epoch - 4ms/step
Epoch 8/100
301/301 - 1s - loss: 0.0021 - 1s/epoch - 4ms/step
Epoch 9/100
301/301 - 1s - loss: 0.0022 - 1s/epoch - 4ms/step
Epoch 10/100
301/301 - 1s - loss: 0.0019 - 961ms/epoch - 3ms/step
Epoch 11/100
301/301 - 1s - loss: 0.0019 - 916ms/epoch - 3ms/step
Epoch 12/100
301/301 - 1s - loss: 0.0018 - 922ms/epoch - 3ms/step
Epoch 13/100
301/301 - 1s - loss: 0.0019 - 961ms/epoch - 3ms/step
Epoch 14/100
301/301 - 1s - loss: 0.0017 - 949ms/epoch - 3ms/step
Epoch 15/100
301/301 - 1s - loss: 0.0018 - 928ms/epoch - 3ms/step
Epoch 16/100
301/301 - 1s - loss: 0.0018 - 91

In [6]:
# Create a line graph of the actual stock prices and the predicted prices
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df_pred['Actual'], name='Actual'))
fig.add_trace(go.Scatter(x=df.index, y=df_pred['Train Predicted'], name='Train Predicted'))
fig.add_trace(go.Scatter(x=df.index, y=df_pred['Test Predicted'], name='Test Predicted'))
fig.update_layout(title='Apple Stock Price Prediction (Last 3 Months)',
                  yaxis_title='Price ($)',
                 xaxis_rangeslider_visible=True)
pyo.iplot(fig)